In [ ]:
from torch.utils.data import DataLoader

from tqdm import tqdm

import torchvision.transforms as transforms
import torchvision

import torch.nn.functional as F
import torch.optim as optim
import torch.nn as nn 
import torch

import matplotlib.pyplot as plt
import numpy as np

BATCH_SIZE = 32
LEARNING_RATE = 1e-3
EPOCH = 50

In [ ]:
device = torch.device('mps')

# Dataset

In [ ]:
transforms = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset   = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transforms)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle=True)

testset    = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transforms)
testloader = torch.utils.data.DataLoader(testset, batch_size=32, shuffle=True)

# Modeling

In [ ]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, strides=1):
        super(ResidualBlock, self).__init__()
        self.strides = strides
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.conv_block = nn.Sequential(
            nn.Conv2d(self.in_channels, self.out_channels, kernel_size=3, stride=strides, padding=1, bias=False),
            nn.BatchNorm2d(self.out_channels),
            nn.ReLU(),

            nn.Conv2d(self.out_channels, self.out_channels, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(self.out_channels)
        )

        if self.strides != 1 or self.in_channels != self.out_channels:
            self.downsample = nn.Sequential(
                nn.Conv2d(self.in_channels, self.out_channels, kernel_size=1, stride=strides, bias=False),
                nn.BatchNorm2d(self.out_channels)
            )

    def forward(self, x):
        out = self.conv_block(x)
        if self.strides != self.in_channels != self.out_channels:
            x = self.downsample(x)

        out = F.relu(x + out)
        return out

In [ ]:
class ResNet(nn.Module):
    def __init__(self, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_channels = 64
        self.base = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU()
        )

        self.layer1 = self._make_layer(64,  num_blocks[0], strides=1)
        self.layer2 = self._make_layer(128, num_blocks[1], strides=2)
        self.layer3 = self._make_layer(256, num_blocks[2], strides=2)
        self.layer4 = self._make_layer(512, num_blocks[3], strides=2)
        self.gap    = nn.AvgPool2d(4)
        self.fc     = nn.Linear(512, num_classes)

    def _make_layer(self, out_channels, num_blocks, strides):
        strides = [strides] + [1] * (num_blocks - 1)
        layers = []

        for stride in strides:
            block = ResidualBlock(self.in_channels, out_channels, stride)
            layers.append(block)
            self.in_channels = out_channels

        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.base(x)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.gap(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

def modeltype(model):
    if model == 'resnet18':
        return ResNet([2, 2, 2, 2])

    elif model == 'resnet34':
        return ResNet([3, 4, 6, 3])

In [ ]:
resnet = modeltype('resnet18').to(device)
print(resnet)

critetion = nn.CrossEntropyLoss()
optimizer = optim.Adam(resnet.parameters(), lr=LEARNING_RATE)

In [ ]:
PATH = './models/cifar_resnet.pth'

In [ ]:
loss_ = list()
n = len(trainloader)

for epoch in range(EPOCH):
    running_loss = 0.0
    for data in trainloader:
        inputs, labels = data[0].to(device), data[1].to(device)
        optimizer.zero_grad()
        outputs = resnet(inputs)
        loss = critetion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    loss_.append(running_loss / n)
    print("[%d] loss : %.3f" % (epoch + 1, running_loss / len(trainloader)))